# Data Integration & Preprocessing for Customer Satisfaction Analysis

This notebook is part of a larger project exploring customer satisfaction in Brazilian e-commerce using the [Olist dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce).  
It builds on the cleaned datasets prepared in the previous notebook by joining them into a single analytical table, performing additional data cleaning, and engineering features to support the upcoming analysis.


**Goals of this notebook:**

- Join the cleaned tables into a consolidated dataset at the order level  
- Clean and standardize the merged dataset (e.g., data types, missing values)  
- Create new features to capture relevant aspects of orders, products, payments etc.
- Export the final dataset for analysis

**This notebook is preceded and followed by:**

- [Data Cleaning Notebook](./01_data_cleaning.ipynb): loads and prepares the individual raw datasets for integration  
- [Exploratory Analysis Notebook](./03_customer-satisfaction-analysis.ipynb): investigates customer satisfaction patterns and key influencing factors
---

## **Structure of the Notebook**

> _Note: Section links and “Back to top” links work best in Jupyter environments (e.g., Jupyter Lab or VS Code). They may not work as expected when clicked directly on GitHub._

- [Data Integration](#data-integration)
  - [Loading cleaned datasets](#loading-cleaned-csv-files-into-duckdb-tables)
  - [Joining tables into order-level dataset](#joining-tables-into-order-level-dataset)

In [47]:
import duckdb
import os
import sys
sys.path.append(os.path.abspath(".."))
import utils as u
import pandas as pd

## **Data Integration**

### Loading Cleaned CSV Files into DuckDB Tables

In [48]:
# Path to the folder containing cleaned CSV files
data_path = "../data/cleaned"

# Connect to an in-memory DuckDB database
con = duckdb.connect(database=":memory:")

# List of table names to load
tables = [
    "customers",
    "orders",
    "items",
    "payments",
    "reviews",
    "products",
    "sellers",
]

# Create and load each table from CSV
for table in tables:
    con.execute(f"""
        CREATE TABLE {table} AS 
        SELECT * FROM read_csv_auto('{data_path}/{table}.csv');
    """)

[🠉 Back to top](#structure-of-the-notebook)

### Joining Tables into Order-Level Dataset

To understand what drives **customer satisfaction**, we need a dataset that captures the entire customer experience — including products purchased, delivery timing, seller location, payment methods, and reviews.  
Since **reviews** are provided **for whole orders** rather than individual products, all features must be aggregated or merged to the **order level** to allow meaningful analysis.

To build this dataset, we:

- **Join** cleaned transactional tables — `items`, `products`, `sellers`, `orders`, `payments`, `reviews`, and `customers` — to gather all relevant data per order  
- **Aggregate** product-level data to the order level (e.g., summing prices, averaging product dimensions)
- **Capture variability** in product categories, sellers, and states:  
  - Orders with more than one product category are labeled `"multiple_categories"`  
  - Similarly, `"multiple_sellers"` and `"multiple_states"` indicate mixed-origin orders  
- **Summarize payment behavior** by counting payment records and installments per order  
- **Filter** the data to include only **delivered** orders with a **non-null review score**, ensuring we focus only on completed and rated transactions

This ensures that **each row corresponds to a unique order** and contains a rich set of features to analyze customer satisfaction drivers.

<details>
<summary><strong>How the Order-Level Dataset Was Built (Click to expand)</strong></summary>


**➤ `item_quantities` (Per-Product-Per-Order)**

- Combines `items`, `products`, and `sellers` to generate one row per product within each order.
- Handles nulls using `COALESCE` to label missing product categories or seller info as `'unknown'`.
- Aggregates:
  - `COUNT(*)` for number of times a product was purchased (product quantity)
  - `SUM(price)` and `SUM(freight_value)`
  - `MAX(product_weight_g)` and `MAX(shipping_limit_date)` for key logistics indicators
- Pulls product-level features using `AVG()` (later rounded) for:
  - `product_name_length`, `product_description_length`
  - `product_photos_qty`, `product_length_cm`, `product_height_cm`, `product_width_cm`

---

**➤ Product and Seller Categorization**

**`category_per_order`**
- If all items in an order share the same category → keep it.
- If there are multiple → label as `'multiple_categories'`.

**`seller_state_per_order` and `seller_id_per_order`**
- Same logic: return actual seller or state if unique, otherwise label as `'multiple_sellers'` or `'multiple_states'`.

**`product_features_per_order`**
- Averages and rounds product features at the order level to summarize size, content richness, and complexity of the products in the order.

---

**➤ `orders_items` (Aggregate Order Metrics)**

- Aggregates order content:
  - Number of **unique products** and **total quantity**
  - **Total price**, **freight**, and **total amount**
  - **Total order weight**

---

**➤ `payments_agg` (Payment Profile)**

- Aggregates payment behavior:
  - Max installments
  - Number of payment records
  - Aggregated distinct payment types (as a comma-separated string)

---

**➤ Final Output**

Combines all previous aggregations into a single **order-level** dataset and adds:

- Review score and timing info from `reviews`
- Actual vs. estimated delivery dates from `orders`
- Customer and seller metadata
- Product-level feature summaries

---

**➤ Final Filtering**

Only includes:
- Orders marked as **'delivered'**
- Orders that **have a review score**

</details>

In [49]:
# Build Query to Create Order-Level Dataset
q_orders = """
WITH item_quantities AS (
  -- Aggregate product and seller info per product in each order
  SELECT
    i.order_id,
    i.product_id,
    COALESCE(p.product_category_name_english, 'unknown') AS product_category_name,
    COALESCE(s.seller_state, 'unknown') AS seller_state,
    COALESCE(s.seller_id, 'unknown') AS seller_id,
    COUNT(*) AS product_quantity,
    SUM(i.price) AS product_price,
    SUM(i.freight_value) AS product_freight,
    MAX(p.product_weight_g) AS product_weight_g,
    MAX(i.shipping_limit_date) AS shipping_limit_date,
    AVG(p.product_name_lenght) AS product_name_length,
    AVG(p.product_description_lenght) AS product_description_length,
    AVG(p.product_photos_qty) AS product_photos_qty,
    AVG(p.product_length_cm) AS product_length_cm,
    AVG(p.product_height_cm) AS product_height_cm,
    AVG(p.product_width_cm) AS product_width_cm
  FROM items AS i
  LEFT JOIN products AS p ON i.product_id = p.product_id
  LEFT JOIN sellers AS s ON i.seller_id = s.seller_id
  GROUP BY i.order_id, i.product_id, s.seller_state, s.seller_id, p.product_category_name_english
),

-- Collapse product category into a single value per order
category_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT product_category_name) = 1 
        THEN MAX(product_category_name)
      ELSE 'multiple_categories'
    END AS product_category_name
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse seller state into a single value per order
seller_state_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT seller_state) = 1 
        THEN MAX(seller_state)
      ELSE 'multiple_states'
    END AS seller_state
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse seller ID into a single value per order
seller_id_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT seller_id) = 1 
        THEN MAX(seller_id)
      ELSE 'multiple_sellers'
    END AS seller_id
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse product ID into a single value per order
product_id_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT product_id) = 1 
        THEN MAX(product_id)
      ELSE 'multiple_products'
    END AS product_id
  FROM item_quantities
  GROUP BY order_id
),

-- Average product-level features across items in the order
product_features_per_order AS (
  SELECT
    order_id,
    ROUND(AVG(product_name_length)) AS product_name_length,
    ROUND(AVG(product_description_length)) AS product_description_length,
    ROUND(AVG(product_photos_qty)) AS product_photos_qty,
    ROUND(AVG(product_length_cm)) AS product_length_cm,
    ROUND(AVG(product_height_cm)) AS product_height_cm,
    ROUND(AVG(product_width_cm)) AS product_width_cm
  FROM item_quantities
  GROUP BY order_id
),

-- Aggregate item-level purchase details per order
orders_items AS (
  SELECT
    order_id,
    COUNT(DISTINCT product_id) AS num_unique_products,
    ROUND(SUM(product_quantity)) AS num_items,
    SUM(product_price) AS total_price,
    SUM(product_freight) AS total_freight,
    SUM(product_price + product_freight) AS total_amount,
    SUM(product_quantity * product_weight_g) AS total_order_weight,
    MAX(shipping_limit_date) AS shipping_limit_date
  FROM item_quantities
  GROUP BY order_id
),

-- Aggregate payment details per order
payments_agg AS (
  SELECT
    order_id,
    MAX(payment_installments) AS max_payment_installments,
    COUNT(payment_sequential) AS n_payment_records, 
    STRING_AGG(DISTINCT payment_type, ', ' ORDER BY payment_type) AS payment_types,
    COUNT(DISTINCT payment_type) AS n_payment_types
  FROM payments
  GROUP BY order_id
)

------ Joined Order-Level Table ------
SELECT
  oi.order_id,
  r.review_score,
  pid.product_id,
  cat.product_category_name,
  oi.num_unique_products,
  oi.num_items,
  oi.total_price,
  oi.total_freight,
  oi.total_amount,
  oi.total_order_weight,
  pf.product_name_length,
  pf.product_description_length,
  pf.product_photos_qty,
  pf.product_length_cm,
  pf.product_height_cm,
  pf.product_width_cm,
  pa.payment_types,
  pa.n_payment_types,
  pa.max_payment_installments,
  pa.n_payment_records,
  oi.shipping_limit_date,
  o.order_delivered_carrier_date,
  o.order_delivered_customer_date,
  o.order_estimated_delivery_date,
  r.review_creation_date,
  r.review_answer_timestamp,
  c.customer_unique_id,
  c.customer_state,
  ssoid.seller_id,
  sso.seller_state

FROM orders_items AS oi
LEFT JOIN orders AS o USING(order_id)
LEFT JOIN customers AS c USING(customer_id)
LEFT JOIN payments_agg AS pa USING(order_id)
LEFT JOIN reviews AS r USING(order_id)
LEFT JOIN category_per_order AS cat USING(order_id)
LEFT JOIN seller_state_per_order AS sso USING(order_id)
LEFT JOIN seller_id_per_order AS ssoid USING(order_id)
LEFT JOIN product_features_per_order AS pf USING(order_id)
LEFT JOIN product_id_per_order AS pid USING(order_id)

-- Filter: only delivered orders with valid review scores
WHERE o.order_status = 'delivered' 
  AND r.review_score IS NOT NULL

ORDER BY oi.order_id;
"""

In [50]:
# Execute the final SQL query and convert the result to a pandas DataFrame
df_orders_raw = con.execute(q_orders).df()

# Show shape of the resulting dataset
print(
    f"Order-level joined table has {df_orders_raw.shape[0]} rows and {df_orders_raw.shape[1]} columns."
)

# Preview the first few rows
df_orders_raw.head()

Order-level joined table has 94782 rows and 30 columns.


,order_id,review_score,product_id,product_category_name,num_unique_products,num_items,total_price,total_freight,total_amount,total_order_weight,...,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_creation_date,review_answer_timestamp,customer_unique_id,customer_state,seller_id,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,5,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,1,1.0,58.90,13.29,72.19,650.0,...,2017-09-19 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,2017-09-21,2017-09-22 10:57:03,871766c5855e863f6eccc05f988b23cb,RJ,48436dade18ac8b2bce089ec2a041202,SP
1,00018f77f2f0320c557190d7a144bdd3,4,e5f2d52b802189ee658865ca93d83a8f,pet_shop,1,1.0,239.90,19.93,259.83,30000.0,...,2017-05-03 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,2017-05-13,2017-05-15 11:34:13,eb28e67c4c0b83846050ddfb8a35d051,SP,dd7ddc04e1b6c2c614352b383efe2d36,SP
2,000229ec398224ef6ca0657da4fc703e,5,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,1,1.0,199.00,17.87,216.87,3050.0,...,2018-01-18 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,2018-01-23,2018-01-23 16:06:31,3818d81c6709e39d06b2738a8d3a2474,MG,5b51032eddd242adc84c38acab88f23d,MG
3,00024acbcdf0a6daa1e931b038114c75,4,7634da152a4610f1595efa32f14722fc,perfumery,1,1.0,12.99,12.79,25.78,200.0,...,2018-08-15 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,2018-08-15,2018-08-15 16:39:01,af861d436cfc08b2c2ddefd0ba074622,SP,9d7a1d34a5052409006425275ba1c2b4,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,5,ac6c3623068f30de03045865e4e10089,garden_tools,1,1.0,199.90,18.14,218.04,3750.0,...,2017-02-13 13:57:51,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,2017-03-02,2017-03-03 10:54:59,64b576fb70d441e8f1b2d7d446e483c5,SP,df560393f3a51e74553ab94004ba5c87,PR


In [51]:
# Close the DuckDB connection
con.close()

[🠉 Back to top](#structure-of-the-notebook)

## **Post-Merge Cleaning**

To ensure the original dataset remains unchanged, we create a copy before performing any cleaning operations.

In [52]:
# Create a copy of the raw joined DataFrame
df_orders = df_orders_raw.copy()

### Initial Inspection of Joined Dataset

After joining the individual tables into an order-level dataset, we inspected  the structure and completeness of the resulting DataFrame.

➤ Dataset Structure

In [53]:
# Display the summary of the DataFrame
u.summarize_table_shapes(df_orders)

,name,rows,columns,column_names,duplicates
0,df,94782,30,"[order_id, review_score, product_id, product_c...",0


The joined order-level dataset contains **94,782 rows** and **30 columns**.  
Importantly, there are **no duplicate rows**, ensuring the integrity of each order record in the final dataset.

➤ Column-wise Overview

To better understand the dataset's composition, we now inspect the column-level properties such as data types, missing values, and number of unique entries

In [54]:
# Quick overview of column properties (dtypes, missing values, uniques)
u.column_overview(df_orders)

Df:


,dtype,non_null,missing_n,missing_%,uniques_n,uniques
order_id,object,94782,0,0.000000,94782,"[00010242fe8c5a6d1ba2dd792cb16214, 00018f77f2f..."
review_score,int64,94782,0,0.000000,5,"[5, 4, 1, 3, 2]"
product_id,object,94782,0,0.000000,29848,"[4244733e06e7ecb4970a6e2683c13e61, e5f2d52b802..."
product_category_name,object,94782,0,0.000000,75,"[cool_stuff, pet_shop, furniture_decor, perfum..."
num_unique_products,int64,94782,0,0.000000,8,"[1, 2, 4, 3, 5, 6, 7, 8]"
num_items,float64,94782,0,0.000000,17,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 7.0, 20.0,..."
total_price,float64,94782,0,0.000000,7536,"[58.9, 239.9, 199.0, 12.99, 199.9, 21.9, 19.9,..."
total_freight,float64,94782,0,0.000000,7841,"[13.29, 19.93, 17.87, 12.79, 18.14, 12.69, 11...."
total_amount,float64,94782,0,0.000000,30316,"[72.19, 259.83, 216.87, 25.78, 218.04000000000..."
total_order_weight,float64,93471,1311,1.383174,2827,"[650.0, 30000.0, 3050.0, 200.0, 3750.0, 450.0,..."


----------------------------------------------------------------------------------------------------------------------------------


<details>
<summary><strong>Key Insights — Missing Values & Data Types (Click to expand)</strong></summary>

**Missing Values**

The missing values fall into **three distinct groups**:
1. **Product-related attributes**  
   - `total_order_weight`, `product_name_length`, `product_description_length`, `product_photos_qty`, `product_length_cm`, `product_height_cm`, `product_width_cm`  
   → These values are missing for ~1,300 orders, likely due to unavailable product information or incomplete joins during merging.

2. **Payment-related attributes**  
   - `payment_types`, `n_payment_types`, `max_payment_installments`, `n_payment_records`  
   → A single row has missing values across these columns, possibly due to an unrecorded or failed transaction.

3. **Delivery date columns**  
   - `order_delivered_carrier_date`, `order_delivered_customer_date`  
   → Although the dataset was filtered to include only orders with `order_status = 'delivered'`, a small number of delivery dates are still missing. This could be due to inconsistencies or delays in data recording on the platform.

**Data Types**

Some columns with low cardinality and repeated string values (e.g. `product_category_name`, `payment_types`, `customer_state` or `seller_state`) are currently of `object` dtype.  
To improve memory efficiency and performance in grouping and aggregation tasks, we will convert them to the category dtype where applicable.

</details>

### Handling Missing Values

## ⚠️ TODO: say that in this section we will handle missing values identified in the previous section

➤ Product-related attributes

## ⚠️ TODO: explain what will be done in this section (identified more than 1300 missing values in each column)

In [55]:
# Define relevant product columns with missing values
missing_product_cols = [
    "product_name_length",
    "product_description_length",
    "product_photos_qty",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm",
    "total_order_weight",
]

## ⚠️ TODO: explain why to create a flag for missing product attributes

In [56]:
# Create missing flag: 1 if any of the product-related fields is missing
df_orders.loc[:, "product_details_missing"] = (
    df_orders["product_name_length"].isna().astype(int)
)

## ⚠️ TODO: explain why to choose median for the missing values

In [57]:
# Median imputation
for col in missing_product_cols:
    median_value = df_orders[col].median()
    df_orders.loc[:, col] = df_orders[col].fillna(median_value)

➤ Payment-related attributes

In [58]:
# Define relevant payment columns with missing values
missing_payment_cols = [col for col in df_orders.columns if "payment" in col]

In [59]:
# Inspect rows with missing payment information
df_orders[df_orders[missing_payment_cols].isna().any(axis=1)][[
    "order_id",
    "product_id",
    "product_category_name",
    "num_unique_products",
    "num_items",
    "total_price",
] + missing_payment_cols]


,order_id,product_id,product_category_name,num_unique_products,num_items,total_price,payment_types,n_payment_types,max_payment_installments,n_payment_records
70948,bfbd0f9bdef84302105ad712db648a6c,5a6b04657a4c5ee34285d1e4619a96b4,health_beauty,1,3.0,134.97,None,<NA>,<NA>,<NA>


We found that only one order has missing values in **payment-related columns**. Since the number of affected rows is minimal and there’s no obvious outlier behavior, we impute these missing values using the **mode** of each column. This ensures consistency with the most common payment patterns observed in the dataset.

In [60]:
# Impute missing payment-related values using the mode of each column
for col in missing_payment_cols:
    mode_value = df_orders[col].mode(dropna=True)[0]
    df_orders[col] = df_orders[col].fillna(mode_value)

➤ Delivery date columns

In [61]:
# Define relevant delivery date columns with missing values
missing_delivery_dates_cols = [
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
]

In [62]:
# Inspect rows with missing delivery dates
df_orders[df_orders[missing_delivery_dates_cols].isna().any(axis=1)].sort_values(
    by=["order_delivered_customer_date", "order_delivered_carrier_date"]
)

,order_id,review_score,product_id,product_category_name,num_unique_products,num_items,total_price,total_freight,total_amount,total_order_weight,...,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_creation_date,review_answer_timestamp,customer_unique_id,customer_state,seller_id,seller_state,product_details_missing
15655,2aa91108853cecb43c84a5dc5b277475,5,44c2baf621113fa7ac95fa06b4afbc68,furniture_decor,1,1.0,179.00,14.98,193.98,7000.0,...,NaT,2017-11-20 19:44:47,2017-11-14,2017-10-17,2017-10-17 10:56:02,a2ac81ecc3704410ae240e74d4f0af40,SP,3f2af2670e104d1bcb54022274daeac5,PR,0
16550,2d1e2d5bf4dc7227b3bfebb81328c15f,5,a50acd33ba7a8da8e9db65094fa990a4,auto,1,1.0,117.30,17.53,134.83,4105.0,...,2017-11-30 18:12:23,NaT,2017-12-18,2017-12-19,2017-12-19 04:15:39,13467e882eb3a701826435ee4424f2bd,SP,8581055ce74af1daba164fdbd55a40de,SP,0
63260,ab7c89dc1bf4a1ead9d6ec1ec8968a84,1,a2a7efc985315e86d4f0f705701b342b,computers_accessories,1,1.0,110.99,9.13,120.12,1133.0,...,2018-06-12 14:10:00,NaT,2018-06-26,2018-06-16,2018-06-16 13:55:00,cce5e8188bf42ffb3bb5b18ff58f5965,SP,ed4acab38528488b65a9a9c603ff024a,SP,0
91015,f5dd62b788049ad9fc0526e3ad11a097,5,2167c8f6252667c0eb9edd51520706a1,industry_commerce_and_business,1,1.0,329.00,25.24,354.24,7750.0,...,2018-06-25 08:05:00,NaT,2018-07-16,2018-06-29,2018-06-29 16:26:37,2f17c5b324ad603491521b279a9ff4de,SP,0bb738e4d789e63e2267697c42d35a2d,SP,0
4937,0d3268bad9b086af767785e3f0fc0133,5,ec165cd31c50585786ffda6feff5d0a6,toys,1,1.0,188.99,15.63,204.62,967.0,...,2018-07-03 09:28:00,NaT,2018-07-24,2018-07-06,2018-07-07 18:48:09,ebf7e0d43a78c81991a4c59c145c75db,SP,8bdd8e3fd58bafa48af76b2c5fd71974,SP,0
17173,2ebdfc4f15f23b91474edf87475f108e,5,e7d5464b94c9a5963f7c686fc80145ad,watches_gifts,1,1.0,139.00,19.07,158.07,275.0,...,2018-07-03 13:57:00,NaT,2018-07-30,2018-07-11,2018-07-11 19:27:46,1bd06a0c0df8b23dacfd3725d2dc0bb9,SP,58f1a6197ed863543e0136bdedb3fce2,MG,0
85498,e69f75a717d64fc5ecdfae42b2e8e086,5,e7d5464b94c9a5963f7c686fc80145ad,watches_gifts,1,1.0,139.00,19.07,158.07,275.0,...,2018-07-03 13:57:00,NaT,2018-07-30,2018-07-07,2018-07-10 11:38:13,3bc508d482a402715be4d5cf4020cc81,SP,58f1a6197ed863543e0136bdedb3fce2,MG,0
12179,20edc82cf5400ce95e1afacc25798b31,5,55bfa0307d7a46bed72c492259921231,books_general_interest,1,1.0,45.90,9.07,54.97,550.0,...,2018-07-03 19:26:00,NaT,2018-07-19,2018-07-06,2018-07-06 20:30:17,175378436e2978be55b8f4316bce4811,SP,343e716476e3748b069f980efbaa294e,SP,0
16719,2d858f451373b04fb5c984a1cc2defaf,5,30b5b5635a79548a48d04162d971848f,sports_leisure,1,1.0,179.00,15.00,194.00,900.0,...,NaT,NaT,2017-06-23,2017-06-25,2017-06-27 01:49:04,d77cf4be2654aa70ef150f8bfec076a6,RS,f9bbdd976532d50b7816d285a22bd01e,SP,0


## ⚠️ TODO: explain why to drop the rows with missing delivery dates

In [63]:
# Drop orders with missing delivery dates, as these likely correspond to undelivered or cancelled orders
df_orders = df_orders.dropna(subset=missing_delivery_dates_cols)

# Reset index after dropping rows to keep DataFrame tidy
df_orders.reset_index(drop=True, inplace=True)